<a href="https://colab.research.google.com/github/cisirtraining/summerschool/blob/master/Day2_LSTM_timeseries_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Mount Google Drive for Data Access**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install scikit-plot

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize


In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU


GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

print('****** GPU Information: ******\n')
printm()

#!kill -9 -1 #kill virtual machine and assign new one

#**Detail Info on LSTM**
Excellent blog post:
[Reference](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

If you interested in maths behind LSTM, read the following (I warn you, think twice ^^):
[Paper](https://arxiv.org/abs/1909.09586)

##Recurrent Neural Networks

The idea behind RNNs is to make use of sequential information. In a traditional neural network we assume that all inputs (and outputs) are independent of each other. But for many tasks that’s a very bad idea. If you want to predict the next word in a sentence you better know which words came before it. RNNs are called recurrent because they perform the same task for every element of a sequence, with the output being depended on the previous computations and you already know that they have a “memory” which captures information about what has been calculated so far.

![](https://drive.google.com/uc?id=1EW9HBneLrcleW1IBR0GE2VJrZhPG-pkc)
## Issues of RNN:
*   Vanishing gradient problem
*   Exploding gradient problem

## Long-Short Term Memory(LSTM)
The LSTM has three gates that update and control the cell states, these are the forget gate, input gate and output gate. The gates use hyperbolic tangent and sigmoid activation functions. 

![alt text](https://drive.google.com/uc?id=1coyG3t6Mbe04WPn8xPdL-64zmdz1oidj)
[Source](https://www.analyticsvidhya.com/blog/2017/12/fundamentals-of-deep-learning-introduction-to-lstm/)

A block has components that make it smarter than a classical neuron and a memory for recent sequences. A block contains gates that manage the block’s state and output. A block operates upon an input sequence and each gate within a block uses the sigmoid activation units to control whether they are triggered or not, making the change of state and addition of information flowing through the block conditional.

There are three types of gates within a unit:

    Forget Gate: conditionally decides what information to throw away from the block.
    Input Gate: conditionally decides which values from the input to update the memory state.
    Output Gate: conditionally decides what to output based on input and the memory of the block.


The forget gate controls what information in the cell state to forget, given new information than entered the network.

![alt text](https://drive.google.com/uc?id=1q-iNGMv_OGIuwpQ7xxtec_wwxo0rhB7j)
[Source](https://medium.com/datadriveninvestor/how-do-lstm-networks-solve-the-problem-of-vanishing-gradients-a6784971a577)

The input gate controls what new information will be encoded into the cell state, given the new input information.
![alt text](https://drive.google.com/uc?id=17432J3U9VTGqsf76LzuH96XwkGjk-aji)
[Source](https://medium.com/datadriveninvestor/how-do-lstm-networks-solve-the-problem-of-vanishing-gradients-a6784971a577)

The output gate controls what information encoded in the cell state is sent to the network as input in the following time step, this is done via the output vector Ht.
![alt text](https://drive.google.com/uc?id=1u7zWxzP__o0lEhuNzvUjufj886KEcAqf)
[Source](https://medium.com/datadriveninvestor/how-do-lstm-networks-solve-the-problem-of-vanishing-gradients-a6784971a577)

##**Dataset Description**

>### Bonn Epileptic Dataset
Source: http://epileptologie-bonn.de/cms/front_content.php?idcat=193&lang=3
 <figure>
<center>
<img src='https://drive.google.com/uc?id=1eIc9eI6ByIt7onVJ2pv6-EQ44qbbTYnT' />
</center>
</figure>



* Bonn dataset is categorized into 5 subsets (A - E), each subset contains 100 single-channel EEG signals of 23.6 seconds obtained from 5 patients - recorded at 173.61 Hz with 12-bit resolution.   
*   Example of raw signal is depicted in the figure with only set E contains epileptic seizure activity.
&nbsp;&nbsp;

<figure>
<center>
<img src='https://drive.google.com/uc?id=1xhvMyPXe_-pkEOwl3Yf0qZkUyiFu6yp4' />
</center>
</figure>



##**Download EEG Dataset**

*  dataset from Bonn University Epilepsy Dataset
*   100 files for each set and 4097 samples per file




In [ ]:
!wget http://epileptologie-bonn.de/cms/upload/workgroup/lehnertz/Z.zip  # Set A
!wget http://epileptologie-bonn.de/cms/upload/workgroup/lehnertz/O.zip  # Set B
!wget http://epileptologie-bonn.de/cms/upload/workgroup/lehnertz/N.zip  # Set C
!wget http://epileptologie-bonn.de/cms/upload/workgroup/lehnertz/F.zip  # Set D
!wget http://epileptologie-bonn.de/cms/upload/workgroup/lehnertz/S.zip  # Set E
  
!unzip Z.zip -d 'Set A'
!unzip O.zip -d 'Set B'
!unzip N.zip -d 'Set C'
!unzip F.zip -d 'Set D'
!unzip S.zip -d 'Set E'
  

##**Load Dataset**

*   Load EEG data from Set A (Normal) and Set E (Epileptic) and their respective labels 
*   Label convention: Normal = 0 and Epileptic = 1



In [ ]:
import glob
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

DATA_DIR_A = './Set A/'
DATA_DIR_B = './Set B/'
DATA_DIR_C = './Set C/'
DATA_DIR_D = './Set D/'
DATA_DIR_E = './Set E/'

LABEL_C1 = 0
LABEL_C2 = 1

def load_data():
    data = []
    nbFiles = 0
    for fname in tqdm(os.listdir(DATA_DIR_A)):
        img = np.loadtxt(DATA_DIR_A + fname)
        data.append([np.array(img), np.array(LABEL_C1)])
        nbFiles+=1
        
    for fname in tqdm(os.listdir(DATA_DIR_E)):
        img = np.loadtxt(DATA_DIR_E + fname)
        data.append([np.array(img), np.array(LABEL_C2)])
        nbFiles+=1
        
    return data


data = load_data()
print('\n')
print('Total Files: {}'.format(len(data)))
print('Shape of data: {}'.format(np.shape(data)))

##**Sanity Check - Visualize Subset from Set A and E**

In [ ]:
# data visualization

import random
mpl.rcParams['figure.figsize'] = (20,10)

fig = plt.figure()

samples_c1 = np.empty((0, 4097), int)
while len(samples_c1) < 4:
    curID = random.randint(0, len(data)) # random id between 0-200
    if data[curID][1] == LABEL_C1:
        samples_c1 = np.vstack((samples_c1, data[curID][0]))
        
samples_c2 = np.empty((0, 4097), int)
while len(samples_c2) < 4:
    curID = random.randint(0, len(data))
    if data[curID][1] == LABEL_C2:
        samples_c2 = np.vstack((samples_c2, data[curID][0]))
        
for i in range(0, 4):
    plt.subplot(421 + i * 2)
    plt.plot(samples_c1[i])
    plt.subplot(421 + i * 2 + 1)
    plt.plot(samples_c2[i])

ax = plt.subplot(421)
ax.set_title("CLASS 1")
ax = plt.subplot(422)
ax.set_title("CLASS 2")

fig.suptitle("Set A vs Set E", fontsize=20)
plt.show()

##**Data Preparation**

*   Split data into training and testing set, 80% training and 20% testing 
*   Convert the label to one-hot-encoded vectors: 0 >> [1,0] and 1 >> [0,1]



In [ ]:
# data preparation
from sklearn.utils import shuffle
data = shuffle(data)
nb_train = round(len(data) * 0.8)
data_train = data[0:nb_train]
data_test = data[nb_train:]

print(data_train[:2])

In [ ]:
X_train = np.array([d[0] for d in data_train])
Y_train = np.array([d[1] for d in data_train])

X_test = np.array([d[0] for d in data_test])
Y_test = np.array([d[1] for d in data_test])

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
print('X_test',X_test)
print('\nY_test',Y_test)


The LSTM network expects the input data (X) to be provided with a specific array structure in the form of:
`[samples, time steps, features]`.

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 4097, 1)
Y_train = Y_train.reshape(Y_train.shape[0], 1)

X_test = X_test.reshape(X_test.shape[0], 4097, 1)
Y_test = Y_test.reshape(Y_test.shape[0], 1)


print('Training data :',X_train.shape)
print('Training label:',Y_train.shape)
print('Testing data  :',X_test.shape)
print('Testing label :',Y_test.shape)

print('\nOriginal training label:\n',Y_train[0:3])


# Encode labels to hot vectors (ex : 2 -> 0=[1,0] and 1=[0,1])
from keras.utils import to_categorical

Y_train = to_categorical(Y_train, num_classes = 2)
Y_test = to_categorical(Y_test, num_classes = 2)


print('\nOne-hot encoded label:\n',Y_train[0:3])

##**Model Training**
>>### Long Short Term Memory (LSTM)

>>![](https://drive.google.com/uc?id=1coyG3t6Mbe04WPn8xPdL-64zmdz1oidj)

[Source](https://www.analyticsvidhya.com/blog/2017/12/fundamentals-of-deep-learning-introduction-to-lstm/)





In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,BatchNormalization, LSTM
from keras.layers import Embedding
from keras.callbacks import EarlyStopping,ModelCheckpoint

batch_size =16
nb_epoch = 25
hidden_size = 128
use_dropout=True

earlyStopping = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='min')
chckpoint = ModelCheckpoint('./drive/My Drive/data/cuDNNlstm_model.h5', save_best_only=True, monitor='val_loss', mode='min')


model = Sequential()
model.add(LSTM(hidden_size, input_shape=(4097,1)))
model.add(Dropout(0.35))
model.add(Dense(2))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mae', 'acc'])
print(model.summary())

history = model.fit(X_train, Y_train, verbose=1, validation_data=(X_test, Y_test),shuffle=True, batch_size=batch_size, epochs=nb_epoch, callbacks=[earlyStopping,chckpoint])

model.save('./drive/My Drive/data/cuDNNlstm_model.h5')


##Plot Learning Curves
Should have something like this!
![alt text](https://drive.google.com/uc?id=1VysuZaAPPOYOvkP-BJ0Wpj-9oKNt8oyF)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(1, figsize = (15,8)) 
    
plt.subplot(221)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid'],loc='lower right') 
    
plt.subplot(222)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid'], loc='upper right') 

plt.show()

print(history.history.keys())

In [ ]:
# Look at confusion matrix 
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (5,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

##**Standard Evaluation Metrics**



$$Precision = \frac{TP}{TP+FP} $$

$$Recall = \frac{TP}{TP+FN} $$

$$F1 = 2 * \frac{Precision*Recall}{Precision + Recall}$$



##**Model Evaluation**

In [ ]:
import itertools
import sklearn
from sklearn.metrics import confusion_matrix
import scikitplot
from keras.models import load_model, Model

# Load trained network
model = load_model('./drive/My Drive/data/cuDNNlstm_model.h5')    # ./drive/My Drive/data/cuDNNlstm_model.h5

print('\n****************** model loaded *******************')

batch_size = 8
# Evaluate the trained model
score = model.evaluate(X_test, Y_test, batch_size=batch_size)
print("\n[info] loss={:.3f}, accuracy: {:.3f}%\n".format(score[0],score[2] * 100))

# Make prediction
y_pred = model.predict(X_test) 
map_characters = {0: 'normal', 1: 'epileptic'}
print('\n', sklearn.metrics.classification_report(np.where(Y_test > 0)[1], np.argmax(y_pred, axis=1), target_names=list(map_characters.values())), sep='')

Y_pred_classes = np.argmax(y_pred,axis = 1) 
Y_true = np.argmax(Y_test,axis = 1) 

# plot confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
plot_confusion_matrix(confusion_mtx, classes = list(map_characters.values()))


# Plot roc curve:
scikitplot.metrics.plot_roc(np.argmax(Y_test, axis=1), y_pred, figsize=(10,8))
plt.show()

##**Visual Sanity Check**

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cv2
from google.colab.patches import cv2_imshow
import random

idx = random.sample(range(0,40), 6) # regenerate random index

for i in range(6):
    (normal, epileptic) = model.predict(X_test)[idx[i]]
    label = "normal" if normal > epileptic else "epileptic"
    proba = epileptic if epileptic > normal else normal
    label = "{}: {:.2f}%".format(label, proba * 100)
    true_label = 'normal' if (Y_test[idx[i]] == 1)[0] else 'epileptic'
   
    plt.subplot(611+i)
    plt.plot(X_test[idx[i]])
    plt.title('True:{} (Predicted:{})'.format(true_label,label))
    plt.axis('on')
    plt.tight_layout()
         

#**LSTM for Regression**

Credit: [Reference](https://machinelearningmastery.com/)

##**Dataset Description**

###International Airline Passengers time-series prediction problem
Problem: Given a year and a month, the task is to predict the number of international airline passengers in units of 1,000. The data ranges from January 1949 to December 1960, or 12 years, with 144 observations.


###**Load Dataset**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

airline = pd.read_csv('/content/drive/My Drive/data/airline-passengers.csv', parse_dates= True, index_col='Month')
airline.head()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(airline)
plt.xlabel('Month')
plt.ylabel('Passengers')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# load the dataset
airline = pd.read_csv('/content/drive/My Drive/data/airline-passengers.csv', usecols=[1])
airline = airline.values
airline = airline.astype('float32')

airline[:10]

LSTMs are sensitive to the scale of the input data, specifically when the sigmoid (default) or tanh activation functions are used. It can be a good practice to rescale the data to the range of 0-to-1, also called normalizing. We can easily normalize the dataset using the MinMaxScaler preprocessing class from the scikit-learn library.

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
airline = scaler.fit_transform(airline)

print('normalized:',airline[:10])
print('max value:',airline.max())
print('min value',airline.min())

plt.figure(figsize=(10,5))
plt.plot(airline)
plt.ylabel('Passengers (Normalized)')
plt.show()

In [ ]:
# split into train and test sets
train_size = int(len(airline) * 0.7)
test_size = len(airline) - train_size
train, test = airline[0:train_size,:], airline[train_size:len(airline),:]

print('training sample:',len(train))
print('test sample:',len(test))



1.   With time series data, the sequence of values are important. 
2.   We will define a function to create new dataset which takes two arguments: the` dataset`, which is a NumPy array that we want to convert into a dataset, and the `look_back`, which is the number of previous time steps to use as input variables to predict the next time period — in this case defaulted to 1.

3. This default will create a dataset where `X` is the number of passengers at a given time (t) and `Y` is the number of passengers at the next time (t + 1)



In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

print('trainX shape:',trainX.shape)
print('trainY shape:',trainY.shape)
print('testX shape:',testX.shape)
print('testY shape:',testY.shape)

The LSTM network expects the input data (X) to be provided with a specific array structure in the form of:
`[samples, time steps, features]`.

Currently, our data is in the form: `[samples, features]` and we are framing the problem as one time step for each sample. We can transform the prepared train and test input data into the expected structure using numpy.reshape() as follows:

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

print('trainX shape:',trainX.shape)
print('testX shape:',testX.shape)

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(16, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
print(model.summary())

history = model.fit(trainX, trainY, epochs=150, batch_size=1, verbose=2)

print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt

plt.figure(1, figsize = (5,5)) 
plt.plot(history.history['loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  

plt.show()


Note that we invert the predictions before calculating error scores to ensure that performance is reported in the same units as the original data (thousands of passengers per month).

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# invert predictions (invert back to normal data form without normalization)
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Because of how the dataset was prepared, we must shift the predictions so that they align on the x-axis with the original dataset. Once prepared, the data is plotted, showing the original dataset in blue, the predictions for the training dataset in orange, and the predictions on the unseen test dataset in green.

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(airline)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(airline)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(airline)-1, :] = testPredict

# plot baseline and predictions
plt.figure(figsize=(10,6))
plt.plot(scaler.inverse_transform(airline))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.xlabel('Month')
plt.ylabel('Passengers (in thousand)')
plt.show()

###**Use specific window size for prediction:**

For example, given the current time (t) we want to predict the value at the next time in the sequence (t+1), we can use the current time (t), as well as the two prior times (t-1) and (t-2) as input variables.

When phrased as a regression problem, the input variables are t-2, t-1, t and the output variable is t+1.


In [ ]:
data = {'X(t)':[112,118,132,129,121], 'X(t-1)':[118,132,129,121,135], 'X(t-2)':[132,129,121,135,148], 'X(t+1)':[129,121,135,148,148]}
df = pd.DataFrame(data)
df